In [1]:
from music21 import converter, instrument, note, chord
import glob
import numpy as np
import keras
import np_utils

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html
Using TensorFlow backend.


In [2]:
notes = []

for file in glob.glob("./../midi_songs/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)

    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
print(len(notes))

57177


In [3]:
pitch_names = sorted(set(item for item in notes))
n_vocab = len(pitch_names)

In [11]:
note_to_int = dict(((number, note) for note, number in enumerate(pitch_names)))
sequence_length = 100
network_input = []
network_output = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
print(n_patterns)

57077


In [12]:
network_input = np.array(network_input)
network_output = np.array(network_output)
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_output = keras.utils.to_categorical(network_output)
print(network_input.shape)
print(network_output.shape)
np.save('./../NumpyDataset/Input-Tensor.npy', network_input)
np.save('./../NumpyDataset/Output-Tensor.npy', network_output)

(57077, 100, 1)
(57077, 358)


In [7]:
network_input = np.load('./../NumpyDataset/Input-Tensor.npy')
network_output = np.load('./../NumpyDataset/Output-Tensor.npy')
network_input = np.reshape(network_input, (57077, 100))
print(network_input.shape, network_output.shape)

(57077, 100) (57077, 358)


In [9]:
n_in = []
for i in range(network_input.shape[0]):
    for j in range(network_input.shape[1]):
        b = []
        a = np.zeros((358))
        note = network_input[i][j]
        a[note] = 1.0
        b.append(a)
    n_in.append(b)
network_inp = np.array(n_in)

In [10]:
network_inp.shape

(57077, 1, 358)